**Dans un premier temps importer son contenu Drive**

---




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importer pandas pour les operations sur les dataframes.

---

In [2]:
import pandas as pd


**transfomer le CSV en dataframe et vérifié son intégrité et que la colonne genres soit une liste**

---

In [3]:
steam_df = pd.read_csv('/content/drive/MyDrive/matthieu/steam_data_sans_langues')
steam_df.head()

,Name,ID,Required age,Gratuit,Price infos,Description détaillée,Site web,Configurations recommandées,Développeur,Éditeur,...,Recommendations,Configurations minimales,Windows,Mac,Linux,coming_soon,date_de_parution,Metacritic score,Metacritic url,Rang
0,Counter-Strike: Global Offensive,730,0,True,Free To Play,Counter-Strike: Global Offensive (CS: GO) expa...,http://blog.counter-strike.net/,NaN,"['Valve', 'Hidden Path Entertainment']",['Valve'],...,'total': 3912429,Minimum:OS: Windows 7/Vista/XPProcessor: Intel...,True,True,True,False,"21 Aug, 2012",83.0,https://www.metacritic.com/game/pc/counter-str...,1
1,Dota 2,570,0,True,Free To Play,"The most-played game on Steam.Every day, milli...",http://www.dota2.com/,NaN,['Valve'],['Valve'],...,'total': 14328,Minimum:OS: Windows 7 or newerProcessor: Dual ...,True,True,True,False,"9 Jul, 2013",90.0,https://www.metacritic.com/game/pc/dota-2?ftag...,2
2,Baldur's Gate 3,1086940,18,False,"'currency': 'EUR', 'initial': 5999, 'final': 5...",Gather your party and return to the Forgotten ...,https://baldursgate3.game,Recommended:Requires a 64-bit processor and op...,['Larian Studios'],['Larian Studios'],...,'total': 247513,Minimum:Requires a 64-bit processor and operat...,True,True,False,False,"3 Aug, 2023",96.0,https://www.metacritic.com/game/pc/baldurs-gat...,3
3,PUBG: BATTLEGROUNDS,578080,0,True,Free To Play,"LAND, LOOT, SURVIVE!Play PUBG: BATTLEGROUNDS f...",https://www.pubg.com,Recommended:Requires a 64-bit processor and op...,"['KRAFTON, Inc.']","['KRAFTON, Inc.']",...,'total': 1667278,Minimum:Requires a 64-bit processor and operat...,True,False,False,False,"21 Dec, 2017",NaN,NaN,4
4,Apex Legends,1172470,0,True,Free To Play,"Conquer with character in Apex Legends, a free...",https://www.ea.com/games/apex-legends,Recommended:Requires a 64-bit processor and op...,['Respawn Entertainment'],['Electronic Arts'],...,'total': 1301,Minimum:Requires a 64-bit processor and operat...,True,False,False,False,"4 Nov, 2020",88.0,https://www.metacritic.com/game/pc/apex-legend...,5


**On retravaille la colonne Genres pour qu'elle soit exploitable par le modele de machine learning TF-IDF**

---


In [4]:
#quelle colonne aurait des valeur null?
print(steam_df.columns[steam_df.isna().any()].tolist())

#et combien de nul y a t'il par colonne?
steam_df.isnull().sum()

['Name', 'Description détaillée', 'Site web', 'Configurations recommandées', 'Développeur', 'Genres', 'Recommendations', 'Configurations minimales', 'date_de_parution', 'Metacritic score', 'Metacritic url']


Name                             57
ID                                0
Required age                      0
Gratuit                           0
Price infos                       0
Description détaillée            92
Site web                       1745
Configurations recommandées    1622
Développeur                      28
Éditeur                           0
Categories                        0
Genres                           22
Recommendations                1362
Configurations minimales         36
Windows                           0
Mac                               0
Linux                             0
coming_soon                       0
date_de_parution                 29
Metacritic score               4464
Metacritic url                 4464
Rang                              0
dtype: int64

In [5]:
steam_df['Genres'].fillna('Genre non renseigné', inplace=True)
steam_df['Description détaillée'].fillna('Description non renseignée', inplace=True)
steam_df['Name'].fillna('nom non renseigné', inplace=True)
steam_df['date_de_parution'].fillna('date de parution non renseigné', inplace=True)

In [6]:
#effacer les espace entre les virgule
steam_df['Genres'] = steam_df['Genres'].str.replace(', ',',')
#comme ça si il y a plusieur mot pour le genre en intercalle '-' avant de séparer par ' '
steam_df['Genres'] = steam_df['Genres'].str.replace(' ','-')
#TFIDF
steam_df['Genres_struct'] = steam_df['Genres'].str.replace(',',' ')
steam_df['Genres_struct']

0       FPS Shooter Multiplayer Competitive Action Tea...
1       Free-to-Play MOBA Multiplayer Strategy eSports...
2       RPG Choices-Matter Character-Customization Sto...
3       Survival Shooter Battle-Royale Multiplayer FPS...
4       Free-to-Play Multiplayer Battle-Royale Shooter...
                              ...                        
6069    Adventure Action RPG Female-Protagonist Fantas...
6070    Strategy RTS Fantasy RPG Multiplayer Singleplayer
6071    Anime Fighting Ninja Action Adventure Multipla...
6072    Indie Action RPG Adventure Sandbox Early-Acces...
6073    Free-to-Play Multiplayer 3D-Platformer Party-G...
Name: Genres_struct, Length: 6074, dtype: object

In [7]:
counts = dict()

for i in steam_df.index:
  #for each element in list (each row, split by ' ', in genres column)
  #-- we're splitting by space so tfidf can interpret the cells
   for g in steam_df.loc[i,'Genres_struct'].split(' '):
#if element is not in counts(dictionary of genres)
      if g not in counts:
        #give genre dictonary entry the value of 1
         counts[g] = 1
      else:
        #increase genre dictionary entry by 1
         counts[g] = counts[g] + 1

#Test Genre Counts
counts.keys()
print(counts['MOBA'])

45


****

**exploitation de la colonne Genre dans le modele de machine learning**

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
!pip install fuzzywuzzy
#https://pypi.org/project/fuzzywuzzy/
from fuzzywuzzy import fuzz

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [9]:
# On crée un objet pour TfidfVectorizer
tfidf_vector = TfidfVectorizer(stop_words='english')
# On applique l'objet à la colonne Genres
# On converti la colonne de Genre en matrice
tfidf_matrix = tfidf_vector.fit_transform(steam_df['Genres_struct'])

In [10]:
tfidf_matrix.shape

(6074, 468)

In [11]:
print(list(enumerate(tfidf_vector.get_feature_names_out())))

[(0, '1980s'), (1, '1990'), (2, '2d'), (3, '360'), (4, '3d'), (5, '40k'), (6, '4x'), (7, '5d'), (8, '6dof'), (9, 'abstract'), (10, 'access'), (11, 'action'), (12, 'addictive'), (13, 'adventure'), (14, 'agriculture'), (15, 'aliens'), (16, 'alternate'), (17, 'ambient'), (18, 'america'), (19, 'american'), (20, 'animation'), (21, 'anime'), (22, 'apocalyptic'), (23, 'arcade'), (24, 'archery'), (25, 'arena'), (26, 'artificial'), (27, 'arts'), (28, 'assassin'), (29, 'asymmetric'), (30, 'asynchronous'), (31, 'atmospheric'), (32, 'attack'), (33, 'atv'), (34, 'audio'), (35, 'auto'), (36, 'automation'), (37, 'automobile'), (38, 'awkward'), (39, 'base'), (40, 'baseball'), (41, 'based'), (42, 'basketball'), (43, 'battle'), (44, 'battler'), (45, 'beat'), (46, 'beautiful'), (47, 'benchmark'), (48, 'bikes'), (49, 'birds'), (50, 'blood'), (51, 'bmx'), (52, 'board'), (53, 'book'), (54, 'boss'), (55, 'bowling'), (56, 'boxing'), (57, 'builder'), (58, 'building'), (59, 'bullet'), (60, 'campaign'), (61, 'ca

In [12]:
# create the cosine similarity matrix
sim_matrix = linear_kernel(tfidf_matrix,tfidf_matrix)
print(sim_matrix)

[[1.         0.44237303 0.1575954  ... 0.24508931 0.0534698  0.1061042 ]
 [0.44237303 1.         0.26322396 ... 0.28914215 0.20987167 0.19216054]
 [0.1575954  0.26322396 1.         ... 0.24248566 0.05107617 0.09338387]
 ...
 [0.24508931 0.28914215 0.24248566 ... 1.         0.14612475 0.18598038]
 [0.0534698  0.20987167 0.05107617 ... 0.14612475 1.         0.05766084]
 [0.1061042  0.19216054 0.09338387 ... 0.18598038 0.05766084 1.        ]]


**Méthodes d'interaction avec steam_df via l'index**


In [13]:
# Convertir l'index en titre
def get_title_from_index(index):
    return steam_df.loc[index, 'Name']

# Convertir le titre en index
def get_index_from_title(title):
    return steam_df[steam_df['Name'] == title].index[0]

# Convertir l'index en année
def get_year_from_index(index):
    return steam_df.loc[index, 'date_de_parution']

# Convertir l'index en plateforme Windows
def get_windows_from_index(index):
    return steam_df.loc[index, "Windows"]

# Convertir l'index en plateforme Mac
def get_mac_from_index(index):
    return steam_df.loc[index, "Mac"]

# Convertir l'index en plateforme Linux
def get_linux_from_index(index):
    return steam_df.loc[index, "Linux"]

# Convertir index en comin_soon
def get_coming_soon_from_index(index):
    return steam_df.loc[index, "coming_soon"]

# Convertir l'index en tags (Genres)
def get_tags_from_index(index):
    return steam_df.loc[index, "Genres"]

# Fonction pour calculer la similarité entre deux titres
def matching_score(a, b):
    return fuzz.ratio(a.lower(), b.lower())

# Trouver le titre le plus proche
def find_closest_title(user_input):
    closest_title = None
    highest_score = 0

    for title in steam_df['Name']:
        score = matching_score(title, user_input)
        if score > highest_score:
            highest_score = score
            closest_title = title

    return closest_title, highest_score


**Correction de fautes de frappes et fait correspondre un titre de jeu**

In [14]:
#find_closest_title propose le titre le similaire à la saisie de l'utilisateur
def closest_names(title):
   leven_scores = list(enumerate(steam_df['Name'].apply(matching_score, b=title)))
   sorted_leven_scores = sorted(leven_scores, key=lambda x: x[1], reverse=True)
   top_closest_names = [get_title_from_index(i[0]) for i in sorted_leven_scores[:10]]  #[['Team Fortress Classic', 'Deathmatch Classic', 'Counter-Strike',~~]
   return top_closest_names

closest_names('trem frotess')

['Team Fortress 2',
 'The Forest',
 'The Witness',
 'Team Fortress Classic',
 'Enemy Front',
 'Due Process',
 'Battle Brothers',
 'Rise of Eros',
 'Demons Roots',
 'RAM Pressure']

**Import de la librairie d'interface**

In [15]:
!pip install gradio
import gradio as gr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.8/395.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 M

ImportError: ignored

In [ ]:
import gradio as gr
import pandas as pd

# Définir la fonction de recommandation basée sur le contenu adaptée à vos données
def gradio_contents_based_recommender(dropdown_option, how_many, min_year, platform, wanted_tags, coming_soon):

    # Retourner le jeu le plus proche en fonction de l'option de liste déroulante
    closest_title, distance_score = find_closest_title(dropdown_option)

    # Créer un DataFrame avec ces en-têtes de colonnes
    recomm_df = pd.DataFrame(columns=['Game Title', 'Release date'])

    # Récupérer l'index correspondant au titre de jeu le plus proche
    games_index = get_index_from_title(closest_title)

    # Récupérer une liste des jeux similaires en tant que tuples (index, score de similarité)
    games_list = list(enumerate(sim_matrix[int(games_index)]))

    # Trier la liste des jeux similaires par score de similarité, du plus élevé au plus bas
    similar_games = list(filter(lambda x: x[0] != int(games_index), sorted(games_list, key=lambda x: x[1], reverse=True)))

    # Afficher le titre de jeu sur lequel est basée la matrice de similarité
    print('Voici la liste des jeux similaires à ' + '\033[1m' + str(closest_title) + '\033[0m' + '.\n')

    # Filtrer les jeux disponibles sur la plateforme sélectionnée
    games_with_platform = []

    if platform != None :
      for i, s in similar_games:
        if platform == "Windows" and get_windows_from_index(i):
          games_with_platform.append((i, s))
        elif platform == "Mac" and get_mac_from_index(i):
          games_with_platform.append((i, s))
        elif platform == "Linux" and get_linux_from_index(i):
          games_with_platform.append((i, s))
    else :
      games_with_platform = similar_games

    # Filtrer les jeux sur le champ coming_soon
    games_with_coming_soon = []
    if coming_soon:
      for i, s in games_with_platform:
        if get_coming_soon_from_index(i):
          games_with_coming_soon.append((i,s))
    else :
      games_with_coming_soon = games_with_platform

    # Filtrer les jeux avec les tags obligatoires sélectionnés par l'utilisateur
    games_with_tags = []
    if wanted_tags != [] :
        for i, s in games_with_coming_soon :
          game_contains_all_tags = True
          for tag in wanted_tags :
            if not tag in get_tags_from_index(i):
              game_contains_all_tags = False
          if game_contains_all_tags :
            games_with_tags.append((i,s))
    else :
      games_with_tags = games_with_coming_soon

    # Transforme l'index en titre
    games_with_name = []
    for i, s in games_with_tags :
      row = {'Game Title': get_title_from_index(i), 'Release date': get_year_from_index(i)}
      games_with_name.append(row)

    # Concaténer les jeux recommandés dans un DataFrame
    recomm_df = pd.concat([recomm_df, pd.DataFrame(games_with_name)], ignore_index=True)

    # Ne conserver que les jeux sortis la même année ou après l'année minimale sélectionnée
    # Step 1: Convert the "Release date" column to string type
    recomm_df['Release date'] = recomm_df['Release date'].astype(str)

    # Step 2: Filter rows where the last 4 characters are digits
    filtered_df = recomm_df[recomm_df['Release date'].str[-4:].str.isdigit()]

    # Step 3: Filter rows where the integer value of "Release date" is >= min_year
    result_df = filtered_df[filtered_df['Release date'].str[-4:].astype(int) >= min_year]
    return result_df[:how_many]

# Créer une liste d'années de calendrier uniques basées sur la colonne 'date_de_parution'
years_sorted = sorted([int(date[-4:]) for date in steam_df['date_de_parution'].unique() if date[-4:].isdigit()])

# Get unique genres
split_genres = steam_df['Genres'].str.split(',')
flat_genres = [genre for genres_list in split_genres for genre in genres_list]
all_tags = list(set(flat_genres))

# Demander à l'utilisateur de saisir le nom du jeu
print("Pour quel jeu souhaitez-vous des recommandations similaires?:")

#Récupérer la demande utilisateur
names = closest_names(input())

# Interface utilisateur avec les éléments d'entrée appropriés
interface = gr.Interface(
    gradio_contents_based_recommender,
    [   gr.Dropdown(names, label="Source de la recommandation"),
        gr.Slider(1, 20, step=int(1),  label="Jeu(x) recommandé(s)"),
        gr.Slider(int(years_sorted[0]), int(years_sorted[-1]), step=int(1), label="Année de sortie minimale"),
        gr.Radio(['Windows', 'Mac', 'Linux'], label="Plateformes"),
        gr.Dropdown( tuple(all_tags), value=["RPG"], multiselect=True, label="Tag(s) obligatoire(s)", info="Choisissez le(s) tag(s) obligatoire(s) sur les jeux à recommander"),
        gr.Checkbox(label="Coming soon", info="Voulez-vous que les jeux recommandés soient des sorties futures"),
    ],
    "dataframe")

interface.launch(debug=True)


